In [1]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 106 kB/s 


In [1]:
import glob
from itertools import chain
import os
import random
import zipfile
from tqdm.notebook import tqdm
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import faiss                            
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
PATH_TRAIN = "/content/drive/MyDrive/COURSES/CS232/CBMIR/ct_dataset_split/train"
PATH_VALID = "/content/drive/MyDrive/COURSES/CS232/CBMIR/ct_dataset_split/val"

In [3]:
class TripletData(Dataset):
    def __init__(self, path, transforms, split="train"):
        self.path = path
        self.split = split    # train or valid
        self.num_cats = 7       # number of categories
        self.transforms = transforms
        self.cats = sorted(os.listdir(path))
    
    def __getitem__(self, idx):
        # our positive class for the triplet
        idx = idx%self.num_cats 
        # print(idx)
        # print(self.path)
        # choosing our pair of positive images (im1, im2)
        positives = os.listdir(os.path.join(self.path, self.cats[idx]))
        im1, im2 = random.sample(positives, 2)
        
        # choosing a negative class and negative image (im3)
        negative_cats = [x for x in range(self.num_cats)]
        negative_cats.remove(idx)
        negative_cat = random.choice(negative_cats)
        negatives = os.listdir(os.path.join(self.path, self.cats[negative_cat]))
        im3 = random.choice(negatives)
        
        im1,im2,im3 = os.path.join(self.path, self.cats[idx], im1), os.path.join(self.path, self.cats[idx], im2), os.path.join(self.path, self.cats[negative_cat], im3)
    
        im1 = self.transforms(pil_loader(im1))
        im2 = self.transforms(pil_loader(im2))
        im3 = self.transforms(pil_loader(im3))
        
        return [im1, im2, im3]
        
    # we'll put some value that we want since there can be far too many triplets possible
    # multiples of the number of images/ number of categories is a good choice
    def __len__(self):
        return self.num_cats*8
    

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Datasets and Dataloaders
train_data = TripletData(PATH_TRAIN, train_transforms)
val_data = TripletData(PATH_VALID, val_transforms)

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=16, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=16, shuffle=False, num_workers=1)

In [4]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [5]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    # Distances in embedding space is calculated in euclidean
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()

# Training

In [ ]:
epochs = 50
device = 'cuda'
best_val_loss = float('inf')
# Our base model
base_model = models.resnet18(pretrained=True).cuda()
model = torch.nn.Sequential(*list(base_model.children())[:-1])
optimizer = optim.Adam(model.parameters(), lr=5e-4)
triplet_loss = TripletLoss()
patience_count = 0
stopping_patience = 10

scheduler = ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=3, threshold=1e-6, threshold_mode='abs', verbose=True)

epsilon_converge = 1e-5

log = {
    'train_loss': [],
    'val_loss': []
}

In [ ]:
# Training
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch, epochs - 1))
    print('-' * 10)
    model.train()  # Set model to training mode
    train_loss = 0.0
    for data in tqdm(train_loader):
        optimizer.zero_grad()
        x1,x2,x3 = data
        e1 = model(x1.to(device))
        e2 = model(x2.to(device))
        e3 = model(x3.to(device)) 
        
        loss = triplet_loss(e1,e2,e3)
        train_loss += loss
        loss.backward()
        optimizer.step()
    print("Train Loss: {}".format(train_loss.item()))

    model.eval()
    val_loss = 0.0
    for data in tqdm(val_loader):
        optimizer.zero_grad()
        x1,x2,x3 = data
        e1 = model(x1.to(device))
        e2 = model(x2.to(device))
        e3 = model(x3.to(device)) 

        loss = triplet_loss(e1,e2,e3)
        val_loss += loss
    print("Val Loss: {}".format(val_loss.item()))

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print('Model saving......................')
        torch.save(model.state_dict(), '/content/drive/MyDrive/COURSES/CS232/CBMIR/model/CBMIR_resnet18_method2.pt')
        patience_count = 0
    else:
        patience_count += 1

    log['train_loss'].append(train_loss)
    log['val_loss'].append(val_loss)

    pickle_out = open(f"/content/drive/MyDrive/COURSES/CS232/CBMIR/log/CBMIR_resnet18_method2.pickle", "wb")
    pickle.dump(log, pickle_out)
    pickle_out.close()

    if patience_count == stopping_patience or val_loss < epsilon_converge:
        break

Epoch 0/49
----------


  0%|          | 0/4 [00:00<?, ?it/s]

Train Loss: 7.528362274169922


  0%|          | 0/4 [00:00<?, ?it/s]

Val Loss: 0.0
Model saving......................


# Load model

In [6]:
base_model = models.resnet18(pretrained=True).cuda()
load_model = torch.nn.Sequential(*list(base_model.children())[:-1])
load_model.load_state_dict(torch.load('/content/drive/MyDrive/COURSES/CS232/CBMIR/model/CBMIR_resnet18_method2.pt'))

<All keys matched successfully>

In [11]:
from torchsummary import summary
summary(load_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
faiss_index = faiss.IndexFlatL2(512)   # build the index
collection = []
# storing the image representations
im_indices = []
with torch.no_grad():
    for f in glob.glob(os.path.join(PATH_TRAIN, '*/*')):
        im = pil_loader(f)
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()

        preds = load_model(im)
        preds = np.array([preds[0].cpu().numpy().flatten()])
        faiss_index.add(preds) #add the representation to index
        im_indices.append(f)   #store the image name to find it later on
    
    for f in glob.glob(os.path.join(PATH_VALID, '*/*')):
        im = pil_loader(f)
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()

        preds = load_model(im)
        preds = np.array([preds[0].cpu().numpy().flatten()])
        faiss_index.add(preds) #add the representation to index
        im_indices.append(f)   #store the image name to find it later on

In [ ]:
pickle_out = open("/content/drive/MyDrive/COURSES/CS232/CBMIR/collection/collection_resnet18_method2.pickle", "wb")
pickle.dump(faiss_index, pickle_out)
pickle_out.close()

In [ ]:
np.save("/content/drive/MyDrive/COURSES/CS232/CBMIR/collection/im_indices_resnet18_method2.npy", im_indices)

# Testing


In [7]:
pickle_in = open("/content/drive/MyDrive/COURSES/CS232/CBMIR/collection/collection_resnet18_method2.pickle", "rb")
load_faiss_index = pickle.load(pickle_in)

In [8]:
load_im_indices = np.load("/content/drive/MyDrive/COURSES/CS232/CBMIR/collection/im_indices_resnet18_method2.npy")

In [ ]:
PATH_TEST = "/content/drive/MyDrive/COURSES/CS232/CBMIR/ct_dataset_split/test"
# Retrieval with a query image
list_ap = []
k = 200
with torch.no_grad():
    for folder in os.listdir(PATH_TEST):
        for f in os.listdir(os.path.join(PATH_TEST, folder)):
            im = pil_loader(os.path.join(PATH_TEST, folder, f))
            print('query:', os.path.join(PATH_TEST, folder, f))
            im = im.resize((224,224))
            im = torch.tensor([val_transforms(im).numpy()]).cuda()

            test_embed = load_model(im)

            test_embed = np.array([test_embed[0].cpu().numpy().flatten()])
            # break
            _, I = load_faiss_index.search(test_embed, k)
            # print("Retrieved Image: {}".format(load_im_indices[I[0][0]]))

            correct = 0
            ap = 0
            for i in range(k):
                pred = load_im_indices[I[0][i]].split('/')[-2]
                print("Retrieved Image:", load_im_indices[I[0][i]])
                if pred == folder:
                  correct += 1
                  ap += correct / (i + 1)
            ap /= correct    
            list_ap.append(ap)
            print('correct:', correct)
            print('ap:', ap)
            print('\n\n/--------------------------------------/')

In [ ]:
import statistics
map = statistics.mean(list_ap)
map